# Create your custom types with Pixano

This notebook will help you create your own custom Python types and their associated PyArrow types, for storing your data exactly how you want to with Pixano.

In [ ]:
import tempfile
from typing import Optional

import pyarrow as pa
import pyarrow.parquet as pq
from pydantic import BaseModel

from pixano.core import BBox, BBoxType, PixanoType, createPyArrowType

## 1. Create a custom type

- Use `pixano.core.PixanoType` and `pydantic.BaseModel` as parent classes
- Define your attributes and methods
- Override the `to_struct` method to define what to export inside your PyArrow format dataset files

Note that PixanoType provides you with generic `from_dict` and `to_dict` methods based on the fields you define in `to_struct`.


In [ ]:
class Camera(PixanoType, BaseModel):
    """Camera type

    Attributes:
        depth_scale (float): Depth scale
        cam_K (list[float]): Camera matrix K
        cam_R_w2c (list[float], optional): 3*3 orientation matrix
        cam_t_w2c (list[float], optional): 3*1 translation matrix
    """

    depth_scale: float
    cam_K: list[float]
    cam_R_w2c: Optional[list[float]]
    cam_t_w2c: Optional[list[float]]

    def __init__(
        self,
        depth_scale: float,
        cam_K: list[float],
        cam_R_w2c: list[float] = None,
        cam_t_w2c: list[float] = None,
    ):
        """Initialize Camera

        Args:
            depth_scale (float): Depth scale
            cam_K (list[float]): Camera matrix K
            cam_R_w2c (list[float], optional): 3*3 orientation matrix. Defaults to None.
            cam_t_w2c (list[float], optional): 3*1 translation matrix. Defaults to None.
        """

        # Define public attributes through Pydantic BaseModel
        super().__init__(
            depth_scale=depth_scale,
            cam_K=cam_K,
            cam_R_w2c=cam_R_w2c,
            cam_t_w2c=cam_t_w2c,
        )

    @staticmethod
    def to_struct() -> pa.StructType:
        """Return Camera type as PyArrow Struct

        Returns:
            pa.StructType: Custom type corresponding PyArrow Struct
        """

        return pa.struct(
            [
                pa.field("depth_scale", pa.float64()),
                pa.field("cam_K", pa.list_(pa.float64())),
                pa.field("cam_R_w2c", pa.list_(pa.float64())),
                pa.field("cam_t_w2c", pa.list_(pa.float64())),
            ]
        )

Test your newly created type to make sure it works:

In [ ]:
my_camera = Camera.from_dict({
    "cam_K": [
        1758.377685546875,
        0.0,
        360.0000000121072,
        0.0,
        1781.137258093513,
        269.9999999622624,
        0.0,
        0.0,
        1.0,
    ],
    "cam_R_w2c": [
        -0.8058909773826599,
        -0.5643280148506165,
        -0.17909124493598938,
        -0.5611616969108582,
        0.8244928121566772,
        -0.0728636085987091,
        0.18877841532230377,
        0.04177902266383171,
        -0.9811305999755859,
    ],
    "cam_t_w2c": [
        -10.521206855773926,
        40.88941192626953,
        1092.1990966796875,
    ],
    "depth_scale": 0.1,
})

print(my_camera.to_dict())

## 2. Create the associated PyArrow type

- Thanks to the `to_struct` method you just defined, all you have to do is call `createPyArrowType` to get your custom PyArrow type.

In [ ]:
CameraType = createPyArrowType(Camera.to_struct(), 'Camera', Camera)

Try creating a PyArrow Array with your type to make sure it works:

In [ ]:
my_camera_array = CameraType.Array.from_list([my_camera])

print(my_camera_array)

## 3. Use your type

Here is a list of things you can do to use your types and make sure everything is setup properly.

Define a schema using your custom type alongside other Pixano types:

In [ ]:
schema = pa.schema([pa.field("Camera", CameraType), pa.field("Bbox", BBoxType)])

Save your arrays of data in a PyArrow table:

In [ ]:
bbox_array = BBoxType.Array.from_list([BBox.from_xywh([1, 2, 3, 4])])

table = pa.Table.from_arrays([my_camera_array, bbox_array], schema=schema)

Save the table as a parquet file and read it back into memory:

In [ ]:
with tempfile.NamedTemporaryFile(suffix=".parquet") as temp_file:
    temp_file_path = temp_file.name
    pq.write_table(table, temp_file_path, store_schema=True)
    table_read = pq.read_table(temp_file_path)

Convert that table back to Python types:

In [ ]:
table_read.to_pylist()

print(table_read.to_pylist()[0]['Bbox'])
print(table_read.to_pylist()[0]['Camera'])